In [115]:
from PIL import Image, ImageDraw 
from IPython.display import Image as DImage
import sys, os, hashlib

In [209]:
mode = ''#pixelhunt
black_mode="ignore" #ingore - only black pixels count
bin_ratio = 200

if not os.path.exists(os.getcwd()+'/examples/'):
    os.mkdir('examples')
if not os.path.exists(os.getcwd()+'/binary/'):
    os.mkdir('binary')
examples_path = os.getcwd()+'/examples/'
binary_path = os.getcwd()+'/binary/'

names = []
values = []
distances = []
RESULT = ''

In [ ]:
input_address = input('Input image address:') #input must be in ""

In [322]:
def binarization(input_img_addr):
    input_img = Image.open(input_img_addr) 
    draw = ImageDraw.Draw(input_img)
    input_width = input_img.size[0]
    input_height = input_img.size[1]
    input_pixels = input_img.load()
    if ".gif" not in input_img_addr:
        for i in range(input_width):
            for j in range(input_height):
                r = input_pixels[i, j][0]
                g = input_pixels[i, j][1]
                b = input_pixels[i, j][2]
                s = r + g + b
                if (s > bin_ratio):
                    r, g, b = 255, 255, 255
                else:
                    r, g, b = 0, 0, 0
                draw.point((i, j), (r, g, b))
    input_img_addr = input_img_addr.replace('.jpg','.png')
    input_img_addr = input_img_addr.replace('.gif','.png')
    if "/examples/" not in input_img_addr:
        input_img_addr = os.getcwd()+'/binary/'+input_img_addr
    else:
        input_img_addr = input_img_addr.replace('/examples/','/binary/')
    input_img.save(input_img_addr, "PNG")
    del draw

In [323]:
def gethash(input_img_addr):
    input_img = Image.open(input_img_addr)
    return (hashlib.md5(input_img.tobytes()).hexdigest())

In [333]:
def gethash2(input_img_addr):
    input_img = Image.open(input_img_addr)
    input_width = input_img.size[0]
    input_height = input_img.size[1]
    input_pixels = input_img.load()
    s = 0
    for i in range(input_width):
            for j in range(input_height):
                s = s + input_pixels[i, j][0]
    print (hash(s/input_width*input_height))
    return hash(s/input_width*input_height)

In [334]:
def hamdist(str1, str2, prevMin=None):
    diffs = 0
    if len(str1) != len(str2):
        return max(len(str1),len(str2))
    for ch1, ch2 in zip(str1, str2):
        if ch1 != ch2:
            diffs += 1
            if prevMin is not None and diffs>prevMin:
                return None
    return diffs

In [335]:
def hamdist2(str1, str2, prevMin=None):
    if str1 > str2:
        return str1 - str2
    else:
        return str2 - str1

In [336]:
def resize(input_img_addr, basewidth):
    img = Image.open(input_img_addr)
    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize))#,Image.ANTIALIAS)
    img.save(input_img_addr) 

In [337]:
def compare1to1(input_img_addr,example_addr):
    example_addr = example_addr.replace('.jpg','.png')
    example_addr = example_addr.replace('.gif','.png')
    input_img_addr = input_img_addr.replace('.jpg','.png')
    input_img_addr = input_img_addr.replace('.gif','.png')
    input_img_addr = os.getcwd()+'/binary/'+input_img_addr
    input_img = Image.open(input_img_addr)
    input_pixels = input_img.load()
    example = Image.open(example_addr)

    true = 0 
    false = 0
    black = 0
    pixels = 0
    
    if mode == 'pixelhunt':
        for i in xrange(0, input_img.size[0]):
                for j in xrange(0, input_img.size[1]):
                    if input_pixels[i, j][0] == 255:
                        black+=1
                    input_pixel = input_img.getpixel((i, j))
                    example_pixel = example.getpixel((i, j))
                    if input_pixel != example_pixel:                        
                        false+=1
                    else:
                        if black_mode == 'ignore' and input_pixels[i, j][0] == 0:
                            break
                        true+=1     
        example_addr = example_addr.replace('.jpg','')
        example_addr = example_addr.replace('.gif','')
        example_addr = example_addr.replace('.png','')
        example_addr = example_addr.replace(os.getcwd()+'/binary/','')
        names.append(example_addr)   
        values.append(true)
        
    else:
        basewidth = 8
        resize(input_img_addr, basewidth)
        input_hash = gethash2(input_img_addr)
        print(input_img_addr,input_hash)
        DImage(input_img_addr)
        resize(example_addr, basewidth)
        example_hash = gethash2(example_addr)
        print(example_addr,example_hash)
        DImage(example_addr)
        distances.append(hamdist2(input_hash, example_hash))
        print(hamdist2(input_hash, example_hash))

In [338]:
def compare1toN(input_img_addr):
    global names
    global values
    global distances
    names = []
    values = []
    distances = []
    examples = os.listdir(examples_path)
    for i in range(len(examples)):
        binarization(examples_path+examples[i])
        compare1to1(input_img_addr,binary_path+examples[i])
            
    dmin=distances[0]
    for i in range(len(distances)):
        if distances[i] is not None and distances[i] < dmin:
            dmin = distances[i]
    print(dmin)

In [339]:
def print1toN():
    for i in range(len(names)):
        print(names[i],values[i])

In [340]:
def result1toN():
    global RESULT 
    RESULT = RESULT + names[values.index(max(values))]
    return names[values.index(max(values))]

In [341]:
binarization(input_address)
compare1toN(input_address)
#print1toN()
#print(result1toN())

17496
('/home/cel/Desktop/UJ/Python/binary/test.png', 17496)
17496
('/home/cel/Desktop/UJ/Python/binary/test.png', 17496)
0
17496
('/home/cel/Desktop/UJ/Python/binary/test.png', 17496)
13770
('/home/cel/Desktop/UJ/Python/binary/ctest.png', 13770)
3726
17496
('/home/cel/Desktop/UJ/Python/binary/test.png', 17496)
18360
('/home/cel/Desktop/UJ/Python/binary/compare.png', 18360)
864
17496
('/home/cel/Desktop/UJ/Python/binary/test.png', 17496)
17496
('/home/cel/Desktop/UJ/Python/binary/a.png', 17496)
0
0
